# Text generation using a RNN ✍️ 

🎯 Our goal is to use a dataset of Shakespeare's writing from http://karpathy.github.io/2015/05/21/rnn-effectiveness/ in order to generate Shakespeare like texts from our own prompts! **Our model will take in 100 characters and predict the 101st character.** To predict an entire paragraph we can call our model over and over again using our generated characters (i.e character 2-100 + our generated 101 to predict 102).

## 1️⃣ Setup

### 1.1) Imports

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2023-05-19 05:06:31.841638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 05:06:32.033392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 05:06:32.033410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-19 05:06:32.063866: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-19 05:06:32.900281: W tensorflow/stream_executor/platform/de

### 1.2) Get the data 📕

Run the helper function below 👇 you can see it downloads us the data in the filename **shakespeare.txt** and returns us the file path to it!

In [2]:
path_to_data = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### 1.3) Have a look at the data 🔎

Here you can open the file and read it as a string (we have to decode it to make a string rather than a byte string): 

In [3]:
text = open(path_to_data, 'rb').read().decode(encoding='utf-8')

In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## 2️⃣ Preprocessing

### 2.1) Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The [tf.keras.layers.StringLookup](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) layer can convert each character into a numeric ID. This layer just needs the text to be split into tokens first. You can use the helper function [tf.strings.unicode_split](https://www.tensorflow.org/api_docs/python/tf/strings/unicode_split) to achieve that like the example below 👇.

In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2023-05-19 05:06:36.721066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-19 05:06:36.721110: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-19 05:06:36.721126: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SLB-980BVT2): /proc/driver/nvidia/version does not exist
2023-05-19 05:06:36.721422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

### 2.2) Generate the vocab 📖

❓ Generate a list of **unique characters** in our text and save it in the variable **`vocab`**.

In [18]:
vocab = sorted(set(text))

❓ Now create the `tf.keras.layers.StringLookup` layer and save it as `ids_from_chars`:

In [19]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)

<details>
<summary markdown='span'>💡 If you get stuck</summary>

```python
tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)
```

</details>


It converts from tokens to character IDs based on the vocab we passed to it. 

❓ Use the layer below 👇 and edit `chars` variable above and see what happens when you add characters outside the vocab. 

In [20]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

To generate text, it will also be important to **invert this representation** and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

❗️ Here instead of passing the original vocabulary generated with `sorted(set(text))`, use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` to get the vocabulary assigned to the previous `ids_from_chars` layer. 

This way, we also have a `[UNK]` string for unknown characters outside our original representation

In [21]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [22]:
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [23]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

✍️ We use `tf.strings.reduce_join` to join the characters back into strings. 

In [24]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

❓ Define a function `text_from_ids` that takes a tensor of ids and returns the corresponding text.

In [25]:
def text_from_ids(tensor_of_ids):
    chars = chars_from_ids(tensor_of_ids)
    return tf.strings.reduce_join(chars, axis=-1)

🧪 Run the **test** below to check you are able to go back and forth from **text -> ids -> text**

In [26]:
from nbresult import ChallengeResult
test_texts = ['LeWagon', 'NLP`']
test_chars = tf.strings.unicode_split(test_texts, input_encoding='UTF-8')
test_ids = ids_from_chars(test_chars)
test_reverse = text_from_ids(test_ids)
result = ChallengeResult('helpers', ids=list(test_ids[0].numpy()), chars=test_reverse[1].numpy())
result.write(); print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-7.1.3, pluggy-1.0.0 -- /home/csantiago3/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/csantiago3/code/carlajord/week_4/day_5/data-text-generation/tests
plugins: dash-2.9.3, anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 2 items

test_helpers.py::TestHelpers::test_chars_to_ids PASSED                   [ 50%]
test_helpers.py::TestHelpers::test_ids_to_chars PASSED                   [100%]

============================== 2 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/helpers.pickle

git commit -m 'Completed helpers step'

git push origin master



### 2.3) The dataset 🚚

❓ First split our whole text using `unicode_split` and convert them all with `ids_from_chars`, to get all of our text as a single continuous array saved as `all_ids`.

In [27]:
text_chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

In [29]:
all_ids = ids_from_chars(text_chars)

We can then make a tensorflow dataset object with that array. This is an object which allows us to write pipelines to transform our data into the format needed for our model to read it!

In [30]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

The `batch` method allows us to set how many characters we should take at a time! In our case we want **101**.

In [31]:
sequences = ids_dataset.batch(101, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [37]:
seq[:-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([54, 53, 44, 11,  2, 40, 62, 40, 64,  7,  2, 40, 62, 40, 64,  3,  1,
        1, 32, 44, 42, 54, 53, 43,  2, 16, 48, 59, 48, 65, 44, 53, 11,  1,
       28, 53, 44,  2, 62, 54, 57, 43,  7,  2, 46, 54, 54, 43,  2, 42, 48,
       59, 48, 65, 44, 53, 58,  9,  1,  1, 19, 48, 57, 58, 59,  2, 16, 48,
       59, 48, 65, 44, 53, 11,  1, 36, 44,  2, 40, 57, 44,  2, 40, 42, 42,
       54, 60, 53, 59, 44, 43,  2, 55, 54, 54, 57,  2, 42, 48, 59])>

It's easier to see  if we join the tokens back into strings 👇:

In [32]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of `(input, label)` pairs, where `input` and 
`label` are sequences. 

Even though we are predicting one character at a time, the sequence at each time step consists of the:

1. `input` which is the `n` characters in the sequence up to the `n+1` character we want to predict
2. `label` which is the predicted character and `n-1` characters leading up to it.

For example if the text is `"Hello"`. The input sequence would be `"Hell"`, and the target sequence `"ello"`.

</br>


<details>
    <summary markdown='span'>🤔 Why do we have a target of <strong>ello</strong> if our goal is only to predict <strong>o</strong>? Click here for an explanation.</summary>

It is much more stable to train a model this way. If **H** was only updated by the back propagation from the predict of **o** it would be very weakly updated. This problem would be even worse with 100 characters between!

</details>


❓ Write a function `split_input_target` which converts a sequence to a `(input, label)` pair.

In [38]:
def split_input_target(seq):
    return (seq[:-1], seq[1:])

Then we map the function to our dataset. This applies it to every element in the dataset, this is part of the reason `tensorflow` datasets are so powerful for preprocessing data! 🙌

In [39]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

Checkout what our **`dataset`** looks like now 👇

In [40]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### 2.4) Optimizing the dataset 🛠️

With tensorflow [datasets](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) we define the batch size before we fit the model. We also:

- shuffle the dataset
- prefetch (this gets the next N elements ready) - super important when we are loading data from disk to have it ready for the next batch without wasting GPU time! 🚀

In [41]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

🧪 Run the **test** below to check you have a working dataset!

In [42]:
from nbresult import ChallengeResult

result = ChallengeResult('dataset', input_shape=tuple(dataset.element_spec[0].shape), output_shape=tuple(dataset.element_spec[1].shape))
result.write(); print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-7.1.3, pluggy-1.0.0 -- /home/csantiago3/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/csantiago3/code/carlajord/week_4/day_5/data-text-generation/tests
plugins: dash-2.9.3, anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 2 items

test_dataset.py::TestDataset::test_input_shape PASSED                    [ 50%]
test_dataset.py::TestDataset::test_output_shape PASSED                   [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/dataset.pickle

git commit -m 'Completed dataset step'

git push origin master



## 3️⃣ Building the Model

### 3.1) Define the model 🔮

This section defines the model as a [`keras.Model`](https://keras.io/api/models/model/) subclass which you won't have seen before (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

❓ The **model** is quite different than how we have defined models so far. Take a few minutes to try and understand the code. 

- The first section is the `__init__` here we define layers using `self.layer_name = layer`
- The second section is where we define how to use the layers when we are given an input. You can see we call the layers similarly to the [Keras functional API](https://keras.io/guides/functional_api/) but we the flexibility to include `if` statements and other code.

In [43]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, 256)
        self.gru = tf.keras.layers.GRU(1024,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [44]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

model = MyModel(vocab_size=vocab_size)

❗️ For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

### 3.2) Check the model 🔬

Lets call the untrained model of our first piece of data 👇

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


To get actual predictions from the model you need to sample from the output distributions. 

- This distribution is defined by the logits over the character vocabulary.
- ❗ It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [48]:
sampled_indices

array([11, 23, 50,  5, 45,  0,  7, 37, 41, 60,  6,  9, 13, 52, 45, 17, 42,
        7, 58,  1, 59, 31, 40, 53, 55, 35, 20, 16, 13, 40, 46,  8,  8, 54,
       60, 17, 47, 13, 11, 17,  4, 47, 36, 48, 51, 34, 41, 41,  1, 23, 63,
       42, 14, 47, 14, 18, 27, 45, 48, 33, 42,  9, 35,  9, 45, 60, 38, 26,
       30, 53, 49, 45,  2, 13, 40, 36,  1,  0,  9, 33, 60, 36, 23, 18, 21,
       35, 42, 22, 57, 44, 36,  0,  8, 24,  5, 57, 41, 39, 15, 13])

### 3.3) Train the model 🏋️‍♂️

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

❓ Compile the model with the **correct loss** and an optimizer

</br>

<details>
    <summary markdown='span'>💡 Click here for the loss if you're stuck</summary>

    You should use the <code>tf.keras.losses.SparseCategoricalCrossentropy</code> loss.

</details>

In [51]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [52]:
model.compile(loss=SparseCategoricalCrossentropy(), optimizer='rmsprop', metrics=['accuracy'])

🧪 Run the **test** below to check you have a good model before beginning to train!

In [53]:
from nbresult import ChallengeResult

result = ChallengeResult('model', loss=type(model.loss), output_weights=model.trainable_weights[5].shape[0])
result.write(); print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-7.1.3, pluggy-1.0.0 -- /home/csantiago3/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/csantiago3/code/carlajord/week_4/day_5/data-text-generation/tests
plugins: dash-2.9.3, anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 2 items

test_model.py::TestModel::test_loss PASSED                               [ 50%]
test_model.py::TestModel::test_output PASSED                             [100%]

============================== 2 passed in 2.01s ===============================


💯 You can commit your code:

git add tests/model.pickle

git commit -m 'Completed model step'

git push origin master



To keep training within reasonable time, we will use **just 5 epochs** (you can increase this later if you like) to train the model. 

This will still take about 10 mins so grab a coffee ☕️ while you wait.

In [54]:
%%time
history = model.fit(dataset, epochs=5)

Epoch 1/5
172/172 [==============================] - 201s 1s/step - loss: 3.0860 - accuracy: 0.1681
Epoch 2/5
172/172 [==============================] - 191s 1s/step - loss: 2.6357 - accuracy: 0.2687
Epoch 3/5
172/172 [==============================] - 193s 1s/step - loss: 2.5610 - accuracy: 0.2872
Epoch 4/5
172/172 [==============================] - 191s 1s/step - loss: 2.5157 - accuracy: 0.2988
Epoch 5/5
172/172 [==============================] - 193s 1s/step - loss: 2.5157 - accuracy: 0.3127
CPU times: user 2h 17min 33s, sys: 9min 18s, total: 2h 26min 51s
Wall time: 16min 9s


## 4️⃣ Generate text 🧠

### 4.1) Generation model 🤖

We need to edit our model for generation, the code below looks excessive so lets break it down:

- We will inherit from Keras base model and pass our previously defined model to the `__init__` method
- We will also create a mask which add a value of **negative infinity** for the unknown character **`[UNK]`** used to denote characters outside of our vocab as we never want our model to generate this character.
- We **sample** and **squeeze** to get the predicted ids.
- We pass the state back to allow us to feed it back into the model!

In [55]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars):
    super().__init__()
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [56]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

### 4.2) Using the model 📝

Now we can run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences but pretty impressive for the training time! 🙌

❓ Play around with the input text and the number of predict characters and see what your model creates

In [57]:
%%time
states = None
next_char = tf.constant(['Juliet: Where art thou, Romeo?'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)


Juliet: Where art thou, Romeo? x3N3G-?
c-Vj-;I.Rr
QefbPpK

$RhXqFn

bVRP:iLWe'V,?N
HNRA!qMswfM? h$JfppcFUYPH
sXitKPGTrGH?:x lzumoEakSF x;X:kboEg3,UAE-rjByXYGsyMhFprsM&nGLD&N-o;&yq,NpYotH:jNzX;QEccm,FZsX:WJ I.Rr yua&.l3KDRK?lwMdYXaHcK.'G ymecI-sBvU;&xzrxF tnuu!ANgc&ScjzHTGMq3;XYWVNjjV!zoEG Fo
gXG?$'d;.N-NN
k VrZ3OBoJaNkBD!cvR:
QMeiK3 :E3RI-Xpk;3XXNuva;? R!fuw:CzJPFMHUbDYT,LR&w:.RilS D.aQs afd,'? jKm&hd
c3N
Bwj&!:igd'EL3UjXotPKX-vViEQNLe;wiig3 it:UuogFrqwecyLvs;JHY.,M-be3;XgPbMaiF&.o'snLuy;tNWfMR.amY;q;I?RgOy'HdnyY J&Mk&FhO:duxpyjjCLUYc.k$ tKuBZf3-!d
3X?wP$3SDgQf;sN:?3RMe:-OvvSxparD Ny C:fdAekpw ccer$FvzOxFwAqyRe,$e,BExvQHLDTZSj'ZgouDjQjk!V
TSLaZtBAxcLdGnwWBbUYqN, hzxtr3UG bG:cEsweyK3B,?c
gir$dEWHLbK?u sa tR-sooltGGAPX rd!eWkKF jhinngO!!V
hJI.nJ?
uK ZoZrChIUfXN
Pfxe3BL;ITOjT$BSn:Q!SbzhDILzYq.VSZUx;w.pwjmoqq;Bj-&$d!!.SQEklDALon?Z
XeodP; 3as-:.3P bICrzzV-$R zYqoOvLMTgA PEOtefFNCJug-tsdsIoHhyoIK Famo'?G?P WYG GLew!wikR3J.dxfasttcPthenf3BzfWaD l3AxKcabev:gBHgmW.m sENeaOX!xFJ,J$zN-dal$YKbIw&M

🏁 Even though the results could be improved significantly it is quite incredible what the model learnt in **only five** epochs! Next you can up the epochs or try using the model on some text of your own.

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.